In [ ]:
import os
import conllu
#from conllu import parse
import spacy
from spacy.language import Language
spacy.prefer_gpu()

In [ ]:
nlp = spacy.load("grc_proiel_trf")

In [ ]:
# if using vectors
!python -m spacy init vectors grc ../assets/grc_floret_cbow_nn2_xn10_b200k_dim300.floret ../vectors/large --mode floret

In [ ]:
# if making senter
!python -m spacy train ../configs/senter_vec.cfg --output ../training/senter/large --gpu-id 0 --paths.train ../corpus/train/grc_perseus-ud-train.spacy --paths.dev ../corpus/dev/grc_perseus-ud-dev.spacy --paths.vectors ../vectors/large --nlp.lang=grc

In [ ]:
!python -m spacy train ../configs/lemmatizer_vec.cfg --output ../training/lemmatizer/large --gpu-id 0 --paths.train ../corpus/train/lemma_train --paths.dev ../corpus/dev/lemma_dev --paths.vectors ../vectors/large --nlp.lang=grc


In [ ]:
!python -m spacy train ../configs/transformer.cfg --output ../training/transformer/proiel/assembled --gpu-id 0 --paths.train ../corpus/train/grc_proiel-ud-train.spacy --paths.dev ../corpus/dev/grc_proiel-ud-dev.spacy --nlp.lang=grc


In [ ]:
# remove accents definition
import unicodedata
def clean_text(text: str) -> str:
    #Cleans the given text by stripping accents and lowercasing.
    try:
        non_accent_characters = [
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
        or char == '̓'  # Greek coronis
        ]
    # str.lower() works for unicode characters
        return ''.join(non_accent_characters).lower()
    except TypeError:
        return text


In [ ]:
import conllu
from conllu import TokenList
from collections import OrderedDict

def separate_sentences(input_file, output_file):
    # Load the input file
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    # Parse the input file using the conllu library
    sentences = conllu.parse(text)

    # Separate the sentences based on the punctuation
    new_sentences = TokenList()
    print(new_sentences)
    current_sentence = []
    for sentence in sentences:
        for token in sentence:
            current_sentence.append(token)
            print(current_sentence)
            if token["form"] == ".":
                new_sentences.append(current_sentence)
                current_sentence = []
        if current_sentence:
            new_sentences.append(current_sentence)
            current_sentence = []
    print("Number of sentences:", len(new_sentences))
    print(new_sentences)
    print(type(new_sentences))
    # create a conllu file with the new sentences

In [ ]:
import conllu
from conllu import TokenList
from collections import OrderedDict

class MyToken(OrderedDict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self["new_field_1"] = "_"
        self["new_field_2"] = "_"
        self["new_field_3"] = "_"
        self["new_field_4"] = "_"
        self["new_field_5"] = "_"
        self["new_field_6"] = "_"
        self["new_field_7"] = "_"
        self["new_field_8"] = "_"
        self["new_field_9"] = "_"
        self["new_field_10"] = "_"



def separate_sentences(input_file, output_file):
    # Load the input file
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    # Parse the input file using the conllu library
    sentences = conllu.parse(text, token_class=MyToken)
    print(sentences)
    # Separate the sentences based on the punctuation
    new_sentences = TokenList()
    current_sentence = []
    for sentence in sentences:
        for token in sentence:
            if not token:
                continue
            if token["form"] == "" or token["form"].startswith("# newdoc id =") or token["form"].startswith("# sent_id =") or token["form"].startswith("# text = NA"):
                continue
            current_sentence.append(token)
            if token["form"] == ".":
                new_sentences.append(current_sentence)
                current_sentence = []
        if current_sentence and len(current_sentence) > 0:
            new_sentences.append(current_sentence)
            current_sentence = []
    print("Number of sentences:", len(new_sentences))
    print(new_sentences)
    print(type(new_sentences))

    # Save the output to a file
    fields = ["id", "form", "lemma", "upos", "xpos", "feats", "head", "deprel", "deps", "misc", "new_field_1", "new_field_2"]
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(conllu.serialize(new_sentences, fields=fields))

In [ ]:
help(TokenList)

In [ ]:
                    data.write(sentence.serialize())
        data.close()


In [ ]:
separate_sentences("../assets/Lemmatization_training_files/plato_iv.conllu", "../assets/Lemmatization_training_files/plato_iv_new.conllu" )

In [ ]:
separate_sentences("../assets/Lemmatization_training_files/aeschylus.conllu", "../assets/Lemmatization_training_files/aeschylus_new.conllu" )

In [ ]:
from pathlib import Path
Path("../assets/Lemmatization_training_files/lemma_train_NFKD").mkdir(parents=True, exist_ok=True)
Path("../assets/Lemmatization_training_files/lemma_train_NFKC").mkdir(parents=True, exist_ok=True)


In [ ]:
ʼ

In [ ]:
# Normalization to NFKD
import random
import unicodedata
def normalize(text):
    return unicodedata.normalize('NFKD', text)
# # load files from folder

#input_file = "../assets/Lemmatization_training_files/lemma_train_NFKD/aristotle_NFKD.spacy"
#train_output_file = "../assets/Lemmatization_training_files/lemma_train_NFKD/aristotle_NFKD_train.spacy"
#dev_output_file = "../assets/Lemmatization_training_files/lemma_train_NFKD/aristotle_NFKD_dev.spacy"

for file in os.listdir("../assets/Lemmatization_training_files"):
    if file.endswith(".conllu"):
        print(file)
        data = open(
            f"../assets/Lemmatization_training_files/{file}", "r", encoding="utf-8"
        )
        annotations = data.read()
        sentences = conllu.parse(annotations)
        for sentence in sentences:
            for token in sentence:
                if token["form"] in ['', "_", '—', '-']:
                    #print ('Empty form:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["form"] = token["lemma"]
                    #print(clean_text(sentence))
                if token["lemma"] in ['', "_", '—', '-']:
                    #print ('Empty lemma:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["lemma"] = ""                     
                if token["id"] == '':
                    print ('Empty id:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["id"] = "UNK"
                if token["upos"] == '':
                    print ('Empty upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["upos"] = ""
                if token["upos"] in ['END', 'MID']:
                    print ('Wrong upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["upos"] = "NOUN"
                token["form"] = normalize(token["form"])
                token["lemma"] = normalize(token["lemma"])
        # Shuffle the sentences randomly
        random.shuffle(sentences)

        # Split the sentences into train and dev data
        train_data = sentences[:int(len(sentences) * 0.8)]
        dev_data = sentences[int(len(sentences) * 0.8):]
        with open("../assets/Lemmatization_training_files/lemma_train_NFKD/" + "{0}_NFKD_train.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in train_data:
                    data.write(sentence.serialize())
        data.close()
        with open("../assets/Lemmatization_training_files/lemma_train_NFKD/" + "{0}_NFKD_dev.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in dev_data:
                data.write(sentence.serialize())
        data.close()


In [ ]:
# check length of connlu file
for file in os.listdir("../assets/Lemmatization_training_files/lemma_train_NFKD"):
    if file.endswith(".conllu"):
        #print(file)
        data = open(
            f"../assets/Lemmatization_training_files/lemma_train_NFKD/{file}", "r", encoding="utf-8"
        )
        annotations = data.read()
        sentences = conllu.parse(annotations)
        if len(sentences) >= 10:
            print(file, ": ", len(sentences), " sentences")
            # convert conllu to spacy file
            !python -m spacy convert "../assets/Lemmatization_training_files/lemma_train_NFKD/{file}" ../assets/Lemmatization_training_files/lemma_train_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens
        else:
            print(file, ": ", len(sentences), " sentences")
            !python -m spacy convert "../assets/Lemmatization_training_files/lemma_train_NFKD/{file}" ../assets/Lemmatization_training_files/lemma_train_NFKD/ -c conllu -m --merge-subtokens



In [ ]:
# check length of connlu file xenophon_ii_NFKD_train.conllu
data = open(
            f"../assets/Lemmatization_training_files/xenophon_ii.conllu", "r", encoding="utf-8"
        )
annotations = data.read()
sentences = conllu.parse(annotations)
print(len(sentences))
print(sentences[0])


In [ ]:
# Normalization to NFKC
import unicodedata
def normalize(text):
    return unicodedata.normalize('NFKC', text)
# # load files from folder

#input_file = "../assets/Lemmatization_training_files/lemma_train_NFKC/aristotle_NFKC.spacy"
train_output_file = "../assets/Lemmatization_training_files/lemma_train_NFKC/aristotle_NFKC_train.spacy"
dev_output_file = "../assets/Lemmatization_training_files/lemma_train_NFKC/aristotle_NFKC_dev.spacy"

for file in os.listdir("../assets/Lemmatization_training_files"):
    if file.endswith(".conllu"):
        print(file)
        data = open(
            f"../assets/Lemmatization_training_files/{file}", "r", encoding="utf-8"
        )
        annotations = data.read()
        sentences = conllu.parse(annotations)
        for sentence in sentences:
            for token in sentence:
                if token["form"] in ['', "_", '—', '-']:
                    #print ('Empty form:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["form"] = token["lemma"]
                    #token["form"] = token["lemma"]
                    #print(clean_text(sentence))
                if token["lemma"] in ['', "_", '—', '-']:
                    #print ('Empty lemma:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["lemma"] = ""                     
                    #token["lemma"] = token["form"]
                if token["id"] == '':
                    print ('Empty id:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["id"] = "UNK"
                if token["upos"] == '':
                    print ('Empty upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["upos"] = ""
                if token["upos"] in ['END', 'MID']:
                    print ('Wrong upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                    token["upos"] = "NOUN"
                token["form"] = normalize(token["form"])
                token["lemma"] = normalize(token["lemma"])
        # Shuffle the sentences randomly
        random.shuffle(sentences)

        # Split the sentences into train and dev data
        train_data = sentences[:int(len(sentences) * 0.8)]
        dev_data = sentences[int(len(sentences) * 0.8):]
        with open("../assets/Lemmatization_training_files/lemma_train_NFKC/" + "{0}_NFKC_train.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in train_data:
                    data.write(sentence.serialize())
        data.close()
        with open("../assets/Lemmatization_training_files/lemma_train_NFKC/" + "{0}_NFKC_dev.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in dev_data:
                    data.write(sentence.serialize())
        data.close()


In [ ]:
# check length of connlu file
for file in os.listdir("../assets/Lemmatization_training_files/lemma_train_NFKC"):
    if file.endswith(".conllu"):
        #print(file)
        data = open(
            f"../assets/Lemmatization_training_files/lemma_train_NFKC/{file}", "r", encoding="utf-8"
        )
        annotations = data.read()
        sentences = conllu.parse(annotations)
        if len(sentences) >= 10:
            print(file, ": ", len(sentences), " sentences")
            # convert conllu to spacy file
            !python -m spacy convert "../assets/Lemmatization_training_files/lemma_train_NFKC/{file}" ../assets/Lemmatization_training_files/lemma_train_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens
        else:
            print(file, ": ", len(sentences), " sentences")
            !python -m spacy convert "../assets/Lemmatization_training_files/lemma_train_NFKC/{file}" ../assets/Lemmatization_training_files/lemma_train_NFKC/ -c conllu -m --merge-subtokens



In [ ]:
#Normalize and convert Perseus dataset

import unicodedata
def normalize_NFKD(text):
    return unicodedata.normalize('NFKD', text)

# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-Perseus"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-Perseus/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/" + "{0}_NFKD.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKD(token["form"])
                    token["lemma"] = normalize_NFKD(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        
import unicodedata
def normalize_NFKC(text):
    return unicodedata.normalize('NFKC', text)

# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-Perseus"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-Perseus/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/" + "{0}_NFKC.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKC(token["form"])
                    token["lemma"] = normalize_NFKC(token["lemma"])
                data.write(sentence.serialize())
        data.close()

In [ ]:

#Normalize and convert PROEIL dataset

import unicodedata
def normalize_NFKD(text):
    return unicodedata.normalize('NFKD', text)

# Remove accents from the PROIEL files
# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-PROIEL"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-PROIEL/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/" + "{0}_NFKD.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKD(token["form"])
                    token["lemma"] = normalize_NFKD(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        


import unicodedata
def normalize_NFKC(text):
    return unicodedata.normalize('NFKC', text)

# Remove accents from the PROIEL files
# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-PROIEL"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-PROIEL/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/" + "{0}_NKFC.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKC(token["form"])
                    token["lemma"] = normalize_NFKC(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        

In [ ]:
# import Universal dependencies tools validate.py


In [ ]:
# convert conllu to spacy UD_Ancient_Greek and UD_Ancient_Greek-PROIEL
!python -m spacy convert ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/ ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens
!python -m spacy convert ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/ ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens

!python -m spacy convert ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/ ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens
!python -m spacy convert ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/ ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens



# Dataset tests

In [ ]:
lemma_train= DocBin().from_disk('../corpus/train/lemma_train/lemma_train.spacy')
lemma_train_docs = list(lemma_train.get_docs(nlp.vocab))

PROIEL_NFKD= DocBin().from_disk('../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/grc_proiel-ud-train_NFKD.spacy')
PROIEL_NFKD_docs = list(PROIEL_NFKD.get_docs(nlp.vocab))

PROIEL_NFKC= DocBin().from_disk('../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/grc_proiel-ud-train_NKFC.spacy')
PROIEL_NFKC_docs = list(PROIEL_NFKC.get_docs(nlp.vocab))

Perseus_NFKD= DocBin().from_disk('../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/grc_perseus-ud-train_NFKD.spacy')
Perseus_NFKD_docs = list(Perseus_NFKD.get_docs(nlp.vocab))

Perseus_NFKC= DocBin().from_disk('../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/grc_perseus-ud-train_NFKC.spacy')
Perseus_NFKC_docs = list(Perseus_NFKC.get_docs(nlp.vocab))

In [ ]:
# iterate through sentences in lemma_train. If a sentence is in any of the other files, print the sentence and the file it is in
for doc in lemma_train_docs:
    if doc not in PROIEL_NFKD_docs:
        print("PROIEL_NFKD")
        print(doc)
    if doc in PROIEL_NFKC_docs:
        print("PROIEL_NFKC")
        print(doc)
    if doc in Perseus_NFKD_docs:
        print("Perseus_NFKD")
        print(doc)
    if doc in Perseus_NFKC_docs:
        print("Perseus_NFKC")
        print(doc)
        

In [ ]:
#Check sentence in lemma_train contain "XLV." if so, then print sentence
for doc in lemma_train_docs:
    if "XLV." in doc.text:
        print(doc.text)


